# Transform data by using Spark

Apache Spark provides a distributed data processing platform that you can use to perform complex data transformations at scale.


## Load source data

Let's start by loading some historical sales order data into a dataframe.

Review the code in the cell below, which loads the sales order from all of the csv files within the **data** directory. Then click the **&#9655;** button to the left of the cell to run it.

> **Note**: The first time you run a cell in a notebook, the Spark pool must be started; which can take several minutes.

In [2]:
from pyspark import sql
from lib import common_functions
from lib import configuration

In [3]:
spark = common_functions.get_spark_session('dp203')
spark.active()

In [4]:
twenty_n_path = configuration.dp203_input_path+'/2019.csv'
twenty_t_path = configuration.dp203_input_path+'/2020.csv'
twenty_to_path = configuration.dp203_input_path+'/2021.csv'

twenty_n_path

'/home/jovyan/code/files/input/dp203//2019.csv'

In [5]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

orderSchema = StructType([
    StructField("SalesOrderNumber", StringType()),
    StructField("SalesOrderLineNumber", IntegerType()),
    StructField("OrderDate", DateType()),
    StructField("CustomerName", StringType()),
    StructField("EmailAddress", StringType()),
    StructField("Item", StringType()),
    StructField("Quantity", IntegerType()),
    StructField("UnitPrice", FloatType()),
    StructField("TaxAmount", FloatType())
    ])

order_details = spark.read.option("header", True).load(f'{configuration.dp203_input_path}/*.csv', format='csv', schema=orderSchema)
order_details.show()
order_details.count()

+----------------+--------------------+----------+-------------------+--------------------+--------------------+--------+---------+---------+
|SalesOrderNumber|SalesOrderLineNumber| OrderDate|       CustomerName|        EmailAddress|                Item|Quantity|UnitPrice|TaxAmount|
+----------------+--------------------+----------+-------------------+--------------------+--------------------+--------+---------+---------+
|         SO49171|                   1|2021-01-01|      Mariah Foster|mariah21@adventur...|  Road-250 Black, 48|       1|2181.5625|  174.525|
|         SO49172|                   1|2021-01-01|       Brian Howard|brian23@adventure...|    Road-250 Red, 44|       1|  2443.35|  195.468|
|         SO49173|                   1|2021-01-01|      Linda Alvarez|linda19@adventure...|Mountain-200 Silv...|       1|2071.4197| 165.7136|
|         SO49174|                   1|2021-01-01|     Gina Hernandez|gina4@adventure-w...|Mountain-200 Silv...|       1|2071.4197| 165.7136|
|     

32718

## Transform the data structure

The source data includes a **CustomerName** field, that contains the customer's first and last name. Let's modify the dataframe to separate this field into separate **FirstName** and **LastName** fields.

In [6]:
from pyspark.sql.functions import split, col

# Create the new FirstName and LastName fields
transformed_df = order_details.withColumn("FirstName", split(col("CustomerName"), " ").getItem(0)).withColumn("LastName", split(col("CustomerName"), " ").getItem(1))

# Remove the CustomerName field
transformed_df = transformed_df.drop("CustomerName")

transformed_df.show(n=5)

+----------------+--------------------+----------+--------------------+--------------------+--------+---------+---------+---------+---------+
|SalesOrderNumber|SalesOrderLineNumber| OrderDate|        EmailAddress|                Item|Quantity|UnitPrice|TaxAmount|FirstName| LastName|
+----------------+--------------------+----------+--------------------+--------------------+--------+---------+---------+---------+---------+
|         SO49171|                   1|2021-01-01|mariah21@adventur...|  Road-250 Black, 48|       1|2181.5625|  174.525|   Mariah|   Foster|
|         SO49172|                   1|2021-01-01|brian23@adventure...|    Road-250 Red, 44|       1|  2443.35|  195.468|    Brian|   Howard|
|         SO49173|                   1|2021-01-01|linda19@adventure...|Mountain-200 Silv...|       1|2071.4197| 165.7136|    Linda|  Alvarez|
|         SO49174|                   1|2021-01-01|gina4@adventure-w...|Mountain-200 Silv...|       1|2071.4197| 165.7136|     Gina|Hernandez|
|     

The code above creates a new dataframe with the **CustomerName** field removed and two new **FirstName** and **LastName** fields.

You can use the full power of the Spark SQL library to transform the data by filtering rows, deriving, removing, renaming columns, and any applying other required data modifications.

## Save the transformed data

After making the required changes to the data, you can save the results in a supported file format.

> **Note**: Commonly, *Parquet* format is preferred for data files that you will use for further analysis or ingestion into an analytical store. Parquet is a very efficient format that is supported by most large scale data analytics systems. In fact, sometimes your data transformation requirement may simply be to convert data from another format (such as CSV) to Parquet!

Use the following code to save the transformed dataframe in Parquet format (Overwriting the data if it already exists).

In [12]:
transformed_df.write.mode("overwrite").parquet(f'{configuration.dp203_output_path}transformed_data/orders.parquet')
print ("Transformed data saved!")

Transformed data saved!


In the **files** tab (which should still be open above), navigate to the root **files** container and verify that a new folder named **transformed_data** has been created, containing a file named **orders.parquet**. Then return to this notebook.

## Partition data

A common way to optimize performance when dealing with large volumes of data is to partition the data files based on one or more field values. This can significant improve performance and make it easier to filter data.

Use the following cell to derive new **Year** and **Month** fields and then save the resulting data in Parquet format, partitioned by year and month.

In [13]:
from pyspark.sql.functions import year, month, col

dated_df = transformed_df.withColumn("Year", year(col("OrderDate"))).withColumn("Month", month(col("OrderDate")))
dated_df.show(n=5)
dated_df.write.partitionBy("Year","Month").mode("overwrite").parquet(f"{configuration.dp203_output_path}partitioned_data")
print ("Transformed data saved!")

+----------------+--------------------+----------+--------------------+--------------------+--------+---------+---------+---------+---------+----+-----+
|SalesOrderNumber|SalesOrderLineNumber| OrderDate|        EmailAddress|                Item|Quantity|UnitPrice|TaxAmount|FirstName| LastName|Year|Month|
+----------------+--------------------+----------+--------------------+--------------------+--------+---------+---------+---------+---------+----+-----+
|         SO49171|                   1|2021-01-01|mariah21@adventur...|  Road-250 Black, 48|       1|2181.5625|  174.525|   Mariah|   Foster|2021|    1|
|         SO49172|                   1|2021-01-01|brian23@adventure...|    Road-250 Red, 44|       1|  2443.35|  195.468|    Brian|   Howard|2021|    1|
|         SO49173|                   1|2021-01-01|linda19@adventure...|Mountain-200 Silv...|       1|2071.4197| 165.7136|    Linda|  Alvarez|2021|    1|
|         SO49174|                   1|2021-01-01|gina4@adventure-w...|Mountain-20

In the **files** tab (which should still be open above), navigate to the root **files** container and verify that a new folder named **partitioned_data** has been created, containing a hierachy of folders in the format **Year=*NNNN*** / **Month=*N***, each containing a .parquet file for the orders placed in the corresponding year and month. Then return to this notebook.

You can read this data into a dataframe from any folder in the hierarchy, using explicit values or wildcards for partitioning fields. For example, use the following code to get the sales orders placed in 2020 for all months.

In [ ]:
orders_2020 = spark.read.parquet(f'{configuration.dp203_output_path}partitioned_data/Year=2020/Month=*')
orders_2020.show(n=10)

+----------------+--------------------+----------+--------------------+--------------------+--------+---------+---------+---------+----------+
|SalesOrderNumber|SalesOrderLineNumber| OrderDate|        EmailAddress|                Item|Quantity|UnitPrice|TaxAmount|FirstName|  LastName|
+----------------+--------------------+----------+--------------------+--------------------+--------+---------+---------+---------+----------+
|         SO45607|                   1|2020-02-01|louis34@adventure...|    Road-650 Red, 52|       1| 699.0982|  55.9279|    Louis|      Nath|
|         SO45606|                   1|2020-02-01|devon0@adventure-...|    Road-150 Red, 48|       1|  3578.27| 286.2616|    Devon|      Deng|
|         SO45609|                   1|2020-02-01|max8@adventure-wo...|    Road-150 Red, 44|       1|  3578.27| 286.2616|      Max|   Ferrier|
|         SO45608|                   1|2020-02-01|frank26@adventure...|Mountain-100 Blac...|       1|  3374.99| 269.9992|    Frank|   Carlson|

Note that the partitioning columns specified in the file path are omitted in the resulting dataframe.

## Use SQL to transform data

Spark is a very flexible platform, and the **SQL** library that provides the dataframe also enables you to work with data using SQL semantics. You can query and transform data in dataframes by using SQL queries, and persist the results as tables - which are metadata abstractions over files.

First, use the following code to save the original sales orders data (loaded from CSV files) as a table. Technically, this is an *external* table because the **path** parameter is used to specify where the data files for the table are stored (an *internal* table is stored in the system storage for the Spark metastore and managed automatically).

In [ ]:
order_details.write.saveAsTable('sales_orders', format='parquet', mode='overwrite', path='/sales_orders_table')

In the **files** tab (which should still be open above), navigate to the root **files** container and verify that a new folder named **sales_orders_table** has been created, containing parquet files for the table data. Then return to this notebook.

Now that the table has been created, you can use SQL to transform it. For example, the following code derives new Year and Month columns and then saves the results as a partitioned external table.

In [ ]:
sql_transform = spark.sql("SELECT *, YEAR(OrderDate) AS Year, MONTH(OrderDate) AS Month FROM sales_orders")
display(sql_transform.limit(5))
sql_transform.write.partitionBy("Year","Month").saveAsTable('transformed_orders', format='parquet', mode='overwrite', path='/transformed_orders_table')

In the **files** tab (which should still be open above), navigate to the root **files** container and verify that a new folder named **transformed_orders_table** has been created, containing a hierachy of folders in the format **Year=*NNNN*** / **Month=*N***, each containing a .parquet file for the orders placed in the corresponding year and month. Then return to this notebook.

Essentially you've performed the same data transformation into partitioned parquet files as s before, but by using SQL instead of native dataframe methods.

You can read this data into a dataframe from any folder in the hierarchy as before, but because the data files are also abstracted by a table in the metastore, you can query the data directly using SQL.

In [1]:
%%sql

SELECT * FROM transformed_orders
WHERE Year = 2021
    AND Month = 1

UsageError: Cell magic `%%sql` not found.


Because these are *external* tables, you can drop the tables from the metastore without deleting the files - so the transfomed data remains available for other downstream data analytics or ingestion processes.

In [ ]:
%%sql

DROP TABLE transformed_orders;
DROP TABLE sales_orders;